In [1]:
epochs = 50
batch_size = 64
learning_rate = 0.001
n_timesteps = 10 # n. barre del periodo passato per la ricerca di pattern, inclusa ultima data disponibile
giorni_previsione = 1 # n. barre nel futuro di cui si desidera prevedere il prezzo
seed = 10 

def crea_modello(n_timesteps, n_features):
    model = Sequential([
        #BatchNormalization(input_shape=(n_timesteps, n_features)),
        LSTM(50, return_sequences=True, recurrent_initializer="glorot_uniform", kernel_initializer="uniform"),
        LSTM(50, recurrent_initializer="glorot_uniform", kernel_initializer="uniform"),
        Dense(64, activation='relu', kernel_initializer="glorot_uniform"),
        Dense(1, activation='linear', kernel_initializer="glorot_uniform")
    ])
    return model

def to_ft(dati_ticker, giorni_pervisione):
    features = dati_ticker[[
        "Close", 
        "Open", 
        "High",
        "Low",
        "Volume"
    ]]
    target = dati_ticker[["Close"]].shift(-giorni_previsione)
    return features.iloc[:-giorni_previsione], target.iloc[:-giorni_previsione]

from sklearn.preprocessing import PowerTransformer
from sklearn.base import clone
scaler = PowerTransformer()

In [2]:
print("Download dati ticker")
import pandas as pd
import yfinance as yf
import numpy as np
import tensorflow as tf
import fx_com

np.random.seed(seed)
tf.random.set_seed(seed)

pd.set_option("display.max_columns", None)

ticker = yf.download("BTG", start='2010-01-01', end='2019-12-31', progress=False)
ticker.index = ticker.index.date
#ticker = fx.crea_indicatori(ticker)

print("Definizione features e target")
features, target = to_ft(ticker, giorni_previsione)
n_features = features.shape[1]

print("Preparazione array X e Y")
def to_XY(features, target, n_timesteps):
    i_tot = len(features) - n_timesteps + 1
    idx, X, Y = [], [], []
    for i in range(i_tot):
        idx.append(features.index[i + n_timesteps - 1])
        X.append(features.iloc[i:i + n_timesteps])
        Y.append(target.iloc[i + n_timesteps - 1])
    return idx, np.array(X), np.array(Y)

idx, X, Y = to_XY(features, target, n_timesteps)

from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense, BatchNormalization

print("Creazione modello")
model = crea_modello(n_timesteps, n_features)

print("Compilazione modello")
from tensorflow.keras.optimizers import Adam
model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mean_squared_error', metrics=['mae'])

print("Reshape")
n_samples = X.shape[0]
X = X.reshape((n_samples, n_timesteps * n_features))

X_scaler = clone(scaler)
Y_scaler = clone(scaler)
X = X_scaler.fit_transform(X)
Y = Y_scaler.fit_transform(Y)

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test, idx_train, idx_test = train_test_split(X, Y, idx, test_size=0.2, random_state=seed)

print("Reshape ai valori originari")
X_train = X_train.reshape((X_train.shape[0], n_timesteps, n_features))
X_test = X_test.reshape((X_test.shape[0], n_timesteps, n_features))

print(f"Addestramento modello epochs={epochs}, batch_size={batch_size}")
model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, Y_test))

print("Scrittura risultati su file parquet")
import os
test_loss, test_mae = model.evaluate(X_test, Y_test)
if os.path.exists('metriche.parquet'):
    metriche = pd.read_parquet("metriche.parquet")
else:
    metriche = pd.DataFrame()
nuova_riga_metriche = pd.DataFrame([{
    "scaler": type(scaler).__name__,
    "epochs": epochs,
    "eatch_size": batch_size,
    "learning_rate": learning_rate,
    "n_timesteps": n_timesteps,
    "giorni_previsione": giorni_previsione,
    "seed": seed,
    "test_loss": test_loss,
    "test_mae": test_mae
}])
for i, layer in enumerate(model.layers):
    config = layer.get_config()
    nuova_riga_metriche[f"Layer_{i+1}"] = f"{type(layer).__name__}({config['units']})"
metriche = pd.concat([metriche, nuova_riga_metriche], ignore_index=True)
metriche.to_parquet("metriche.parquet")

Download dati ticker
Definizione features e target
Preparazione array X e Y
Creazione modello
Compilazione modello
Reshape
Reshape ai valori originari
Addestramento modello epochs=50, batch_size=32
Epoch 1/50
63/63 [==============================] - 14s 69ms/step - loss: 0.1192 - mae: 0.2396 - val_loss: 0.0519 - val_mae: 0.1726
Epoch 2/50
63/63 [==============================] - 2s 31ms/step - loss: 0.0427 - mae: 0.1553 - val_loss: 0.0392 - val_mae: 0.1488
Epoch 3/50
63/63 [==============================] - 2s 31ms/step - loss: 0.0350 - mae: 0.1419 - val_loss: 0.0313 - val_mae: 0.1319
Epoch 4/50
63/63 [==============================] - 2s 30ms/step - loss: 0.0350 - mae: 0.1413 - val_loss: 0.0300 - val_mae: 0.1286
Epoch 5/50
63/63 [==============================] - 2s 31ms/step - loss: 0.0277 - mae: 0.1228 - val_loss: 0.0307 - val_mae: 0.1352
Epoch 6/50
63/63 [==============================] - 2s 34ms/step - loss: 0.0252 - mae: 0.1192 - val_loss: 0.0224 - val_mae: 0.1081
Epoch 7/50
63/6

In [3]:
metriche

,scaler,epochs,eatch_size,learning_rate,n_timesteps,giorni_previsione,seed,test_loss,test_mae,Layer_1,Layer_2,Layer_3,Layer_4
0,PowerTransformer,50,32,0.001,30,1,10,0.019046,0.103922,None,None,None,None
1,PowerTransformer,50,32,0.001,60,1,10,0.019723,0.098877,None,None,None,None
2,PowerTransformer,50,32,0.001,90,1,10,0.017568,0.098379,None,None,None,None
3,PowerTransformer,50,32,0.001,120,1,10,0.018050,0.095072,None,None,None,None
4,PowerTransformer,50,32,0.001,200,1,10,0.017148,0.097441,None,None,None,None
5,PowerTransformer,50,32,0.001,15,1,10,0.015166,0.090270,None,None,None,None
6,PowerTransformer,50,32,0.001,10,1,10,0.015059,0.086320,None,None,None,None
7,PowerTransformer,50,32,0.001,5,1,10,0.014056,0.089201,None,None,None,None
8,PowerTransformer,50,32,0.001,30,1,10,0.017890,0.098524,None,None,None,None
9,PowerTransformer,50,32,0.001,30,1,10,0.017246,0.094180,"LSTM, units: 200","LSTM, units: 100","Dense, units: 64","Dense, units: 1"


In [4]:
#previsione()
#fx_com.grafico(risultato)

def previsione():
    print("Download dati ticker per previsione")
    dati_previsione = yf.download("BTG", start='2020-01-01', end='2023-12-31', progress=False)

    features_prev, target_prev = to_ft(dati_previsione, giorni_previsione)
    idx_prev, X_prev, Y_prev = to_XY(features_prev, target_prev, n_timesteps)

    X_prev = X_prev.reshape(-1, n_timesteps * n_features)
    X_scaler_prev = clone(scaler)
    Y_scaler_prev = clone(scaler)
    X_prev = X_scaler_prev.fit_transform(X_prev)
    Y_prev = Y_scaler_prev.fit_transform(Y_prev)
    X_prev = X_prev.reshape(-1, n_timesteps, n_features)

    pred = model.predict(X_prev)

    X_prev = X_prev.reshape(-1, n_timesteps * n_features)
    X_prev = X_scaler_prev.inverse_transform(X_prev)
    X_prev = X_prev.reshape(-1, n_timesteps, n_features)

    Y_prev = Y_scaler_prev.inverse_transform(Y_prev)
    pred = Y_scaler_prev.inverse_transform(pred)

    risultato = pd.DataFrame({"Close": X_prev[:, n_timesteps-1, 0], "Target": Y_prev.reshape(-1), "Previsione": pred.reshape(-1).round(2), "Diff": (Y_prev.reshape(-1) - pred.reshape(-1)).round(2)}, index=idx_prev)
